## Общая предобработка данных

In [1]:
import os

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


DATASET_URL = '../data/cbr-press-releases.csv'

Загрузим датасет пресс-релизов, полученных с сайта ЦБ РФ:

In [2]:
df = pd.read_csv(DATASET_URL, parse_dates=['date'])
df.head()

,date,link,title,release,inflation,rate,usd,usd_cur_change_relative,target_categorial,target_absolute,target_relative
0,2013-09-13,http://www.cbr.ru/press/pr/?file=130913_134305...,О процентных ставках по операциям Банка России,Департамент внешних и общественных связей Банк...,6.51,5.5,32.6731,0.994294,0.0,0.0,1.000000
1,2013-10-14,http://www.cbr.ru/press/pr/?file=131014_133001...,О ключевой ставке Банка России,"Пресс-служба Банка России сообщает, что Совет ...",6.14,5.5,32.2663,0.987549,0.0,0.0,1.000000
2,2013-11-08,http://www.cbr.ru/press/pr/?file=131108_133008...,О ключевой ставке Банка России,Совет директоров Банка России 8 ноября 2013 го...,6.27,5.5,32.3803,1.003533,0.0,0.0,1.000000
3,2013-12-13,http://www.cbr.ru/press/pr/?file=131213_133004...,О ключевой ставке Банка России,Совет директоров Банка России 13 декабря 2013 ...,6.50,5.5,32.7518,1.011473,0.0,0.0,1.000000
4,2014-02-14,http://www.cbr.ru/press/pr/?file=14022014_1333...,О ключевой ставке Банка России,Совет директоров Банка России 14 февраля 2014 ...,6.07,7.0,34.8611,1.064403,1.0,1.5,1.272727


`rate` - величина ключевой ставки, которую объявят на следующем заседании. Мы хотели бы знать эту величину. Но мы знаем текущую ставку и эту информацию можно добавить в наши данные сдвинув ставку на один.

In [3]:
df['rate'] = df.rate.shift(1)

В сентябре 2013 года ЦБ ввел ключевую ставку, и назначил ее величину в 5,5% заполним пропущенное значение.

In [4]:
df.loc[0, 'rate'] = 5.5

Дату сделаем индексом, ссылку на пресс-релиз выбросим, эта переменная не поможет в предсказании таргета.

In [5]:
df.set_index('date', inplace=True)
df.drop('link', axis=1, inplace=True)

Для последнего релиза неизвестна целевая переменная (направление изменения ключевой ставки), поэтому исключим его из датасета и сохраним отдельно:

In [6]:
df.sort_values('date', inplace=True)
cur_pr = df.tail(1)
df = df[:-1]

Мы хотим предсказывать направление ставки рефинансирования `target_categorial`, выделим ее как целевую переменную. `target_absolute` и `target_relative` содержит в себе информацию о целевой переменной удалим их из матрицы признаков.

In [7]:
X = df.drop(['target_categorial', 'target_absolute', 'target_relative'], axis=1)
y = df['target_categorial']

Сохраним X и y в файлах, чтобы пользоваться ими во всех экспериментах

In [8]:
directory = './data'

if not os.path.exists(directory):
    os.makedirs(directory)

X.to_csv('./data/x.csv')
y.to_csv('./data/y.csv')

## Минимальный безлайн 

Направление изменения ставки будем предсказывать таким, как в последнем решении. До 13 сентября ключевой ставки не существовало, поэтому мы не можем предсказать таким способом для первого наблюдения, будем оценивать качество бейзлайна без него.

In [9]:
# вероятность 1 для соответствующего класса
def get_proba(y):
    proba = np.zeros((1, 3))
    proba[0, int(y + 1)] = 1.
    return proba

y_pred = y.shift(1)
accuracy = accuracy_score(y[1:], y_pred[1:])
f1 = f1_score(y[1:], y_pred[1:], average='macro')
precision = precision_score(y[1:], y_pred[1:], average='macro')
recall = recall_score(y[1:], y_pred[1:], average='macro')
roc_auc_ovr = roc_auc_score(
    y[1:], 
    np.concatenate(list(map(get_proba, y_pred[1:]))), 
    average='macro', 
    multi_class='ovr'
    )
roc_auc_ovo = roc_auc_score(
    y[1:], 
    np.concatenate(list(map(get_proba, y_pred[1:]))), 
    average='macro', 
    multi_class='ovo')
min_baseline = pd.DataFrame(
    {
        'accuracy': accuracy, 
        'f1': f1, 
        'precision': precision, 
        'recall': recall, 
        'roc_auc_ovr': roc_auc_ovr, 
        'roc_auc_ovo': roc_auc_ovo
    }, 
    index=['min_baseline']
    )

In [10]:
min_baseline

,accuracy,f1,precision,recall,roc_auc_ovr,roc_auc_ovo
min_baseline,0.663158,0.667893,0.669919,0.666111,0.74501,0.749583


Это будет нашим ориетиром в экспериментах. Будем хранить качество наших моделей в `metrics.csv`

In [11]:
min_baseline.to_csv('metrics.csv')